In [1]:
import json
import datetime
import trustedanalytics as ia

ia.connect()

Connected.  This client instance connected to server http://localhost:9099/v1 (version=0.4.3-201511049238) as user test_api_key_1 at 2015-11-10 11:38:52.038901.


## Set up frame...

In [2]:
def add_ids(row):
    my_json = json.loads(row[0])
    patid = my_json['PATID'] if 'PATID' in my_json else None
    visid = my_json['VISID'] if 'VISID' in my_json else None
    return patid, visid

def add_height(row):
    """Function to extract height in inches from json, and add as column in data frame."""
    my_json = json.loads(row[0])
    HEIGHT_INCHES = my_json['HEIGHT_INCHES'] if 'HEIGHT_INCHES' in my_json else 0.0
    return HEIGHT_INCHES

def add_age(row):
    my_json = json.loads(row[0])
    AGE = my_json['AGE'] if 'AGE' in my_json else 0
    return AGE

def add_weight(row):
    my_json = json.loads(row[0])
    WEIGHT_LBS = 150.0
    if 'WEIGHT_LBS' in my_json:
        WEIGHT_LBS = my_json['WEIGHT_LBS']
    try:
        WEIGHT_LBS = ia.float64(WEIGHT_LBS)
    except:
        WEIGHT_LBS = 150.0
    return WEIGHT_LBS

def add_marital_status(row):
    my_json = json.loads(row[0])
    MARITAL_STATUS = my_json['MARITAL_STATUS'] if 'MARITAL_STATUS' in my_json else None
    return MARITAL_STATUS

def add_death_flag(row):
	my_json = json.loads(row[0])
	DEATH_FLAG = my_json['DEATH_FLAG'] if 'DEATH_FLAG' in my_json else None
	return DEATH_FLAG

def add_meds(row):
    my_json = json.loads(row[0])
    med_orders = my_json['MED_ORDER_NAMEs'] if 'MED_ORDER_NAMEs' in my_json else [None]
    discharge_med_orders = my_json['DISCHARGE_MED_ORDER_NAMEs'] if 'DISCHARGE_MED_ORDER_NAMEs' in my_json else [None]
    if med_orders != [None] and discharge_med_orders != [None]:
        meds = med_orders + discharge_med_orders
    elif med_orders == [None]:
        meds = discharge_med_orders
    elif discharge_med_orders == [None]:
        meds = med_orders
    #if med_orders != [None]:
    #    med_orders = [clean_drugs.preprocess(i) for i in med_orders]
    return '|'.join([str(i) for i in med_orders])

def add_marital_status(row):
    my_json = json.loads(row[0])
    MARITAL_STATUS = my_json['MARITAL_STATUS'] if 'MARITAL_STATUS' in my_json else None
    return MARITAL_STATUS

In [3]:
#Load data frame and start fresh...
tutorial_inpat = ia.get_frame("tutorial_inpat_frame")
tutorial_inpat_model = tutorial_inpat.copy()
tutorial_inpat_model.drop_columns([i for i in tutorial_inpat_model.column_names if i != 'data_lines'])
tutorial_inpat_model.add_columns(add_ids, [("PATID", str), ("VISID", str)])
tutorial_inpat_model.add_columns(lambda row: str(row["PATID"]) + str(row["VISID"]), ("EVENT_ID", str))
tutorial_inpat_model.add_columns(add_height, ("HEIGHT_INCHES", ia.float64))
tutorial_inpat_model.add_columns(add_marital_status, ("MARITAL_STATUS", str))
tutorial_inpat_model.add_columns(add_weight, ("WEIGHT_LBS", ia.float64))
tutorial_inpat_model.add_columns(add_death_flag, ("DEATH_FLAG", str))
tutorial_inpat_model.add_columns(add_age, ("AGE", ia.float64))
tutorial_inpat_model.add_columns(add_meds, ("MEDS", str))

[=========================] 100.00% Tasks retries:0 Time 0:00:09
[=========================] 100.00% Tasks retries:0 Time 0:00:30
[=========================] 100.00% Tasks retries:0 Time 0:00:13
[=========================] 100.00% Tasks retries:0 Time 0:00:11
[=========================] 100.00% Tasks retries:0 Time 0:00:14
[=========================] 100.00% Tasks retries:0 Time 0:00:17
[=========================] 100.00% Tasks retries:0 Time 0:00:15
[=========================] 100.00% Tasks retries:0 Time 0:00:17
[=========================] 100.00% Tasks retries:0 Time 0:00:19
[=========================] 100.00% Tasks retries:0 Time 0:00:19


In [4]:
def add_admit_date(row):
    """Function to extract the admit date for a patient"""
    my_json = json.loads(row[0])
    try:
        ADMIT_DATE = my_json['ADM_DATE']['$date']/1000
    except:
        ADMIT_DATE = None
    return ADMIT_DATE

def add_discharge_date(row):
    """Function to extract the discharge date of a patient"""
    my_json = json.loads(row[0])
    try:
        DISCHARGE_DATE = my_json['DISCHARGE_DATE']['$date']/1000
    except:
        DISCHARGE_DATE = None
    return DISCHARGE_DATE

In [5]:
# Add admit and discharge date columns to our data frame...
tutorial_inpat_model.add_columns(add_admit_date, ("ADM_DATE", str))
tutorial_inpat_model.add_columns(add_discharge_date, ("DISCHARGE_DATE", str))

[=========================] 100.00% Tasks retries:0 Time 0:00:18
[=========================] 100.00% Tasks retries:0 Time 0:00:19


In [6]:
tutorial_inpat_model.inspect(columns=[i for i in tutorial_inpat_model.column_names if i != 'data_lines'])

[#]  PATID                 VISID               
[0]  a21ed8168a129a627dc7  ea8c4671999dbfc306e9
[1]  ce2c320507f9afc8a6be  475b875637ad2c3a97f6
[2]  6b5e9b5979e90670ccc4  c87ea2714e1700c18cbd
[3]  3db6c2b132f1e7980be9  bddbbca37fc65b251e95
[4]  193876054eb69bbf5fa7  d83873c4ca458bdcedda
[5]  8a865e88a484cd0eb47d  3d232aded17358de322f
[6]  5b31441e3c35d1912b87  667741db17f135fd04bb
[7]  77ce9838cd87649a3b6c  835ff02eb60c6fcf1793
[8]  062e213248c7513e8aa5  18fc863241929168fa5b
[9]  6fca5d705add2fa21213  a5fe7f2d6717a5ff2162

[#]  EVENT_ID                                  HEIGHT_INCHES  MARITAL_STATUS
[0]  a21ed8168a129a627dc7ea8c4671999dbfc306e9  64.0157470703  S
[1]  ce2c320507f9afc8a6be475b875637ad2c3a97f6  62.0078735352  D
[2]  6b5e9b5979e90670ccc4c87ea2714e1700c18cbd  62.9921264648  M
[3]  3db6c2b132f1e7980be9bddbbca37fc65b251e95  62.9921264648  S
[4]  193876054eb69bbf5fa7d83873c4ca458bdcedda  65.9842529297  S
[5]  8a865e88a484cd0eb47d3d232aded17358de322f  69.0157470703  S
[6]  5b314

#### Filter out missing ids...

In [7]:
# A slightly different syntax for filtering that I find convenient...
# Sometimes dropping data_lines is necessary to do a filter. 
# In these situations, make sure you're done extracting features from the original json...
def filter_patid_nones(row):
	return row['PATID'] is not None 

def filter_visid_nones(row):
	return row['VISID'] is not None 

# Filter out any rows without patid or visid...
tutorial_inpat_model.drop_columns('data_lines')
tutorial_inpat_model.filter(filter_patid_nones)
tutorial_inpat_model.filter(filter_visid_nones)

[=========================] 100.00% Tasks retries:0 Time 0:00:02
[=========================] 100.00% Tasks retries:0 Time 0:00:02
[=========================] 100.00% Tasks retries:0 Time 0:00:01


# Let's D.A.

In [8]:
# Create a working copy...
lts = tutorial_inpat_model.copy()

# Flatten on the separating character...
lts.flatten_column("MEDS", "|")

# We need to create a table ox ID x MED x Count...
lts_lda_input = lts.group_by(["VISID", "MEDS"], ia.agg.count)
if "lts_lda_input_renamed" in ia.get_frame_names():
    ia.drop_frames("lts_lda_input_renamed")
lts_lda_input_renamed = lts_lda_input.copy(name = "lts_lda_input_renamed")

[=========================] 100.00% Tasks retries:0 Time 0:00:02
[=========================] 100.00% Tasks retries:0 Time 0:00:01
[=========================] 100.00% Tasks retries:0 Time 0:00:02
[=========================] 100.00% Tasks retries:0 Time 0:00:01


In [9]:
# Set the parameters for the LDA model...
nTopics = 20
max_iterations = 2

# Clean up any old attempts...
ia.drop_models(['lts_lda_model'])

# Create LDA model..
lda_model =  ia.LdaModel(name='lts_lda_model')
lts_lda_model = lda_model.train(lts_lda_input_renamed, "VISID", "MEDS", 'count', num_topics=nTopics, max_iterations=max_iterations)

# We can extract the per-id topic distributions...
doc_mixes = lts_lda_model['topics_given_doc']

[=========================] 100.00%  Time 0:00:00
[=========================] 100.00% Tasks retries:0 Time 0:00:29
[=========================] 100.00% Tasks retries:0 Time 0:00:01


In [10]:
# Clean up old approaches...
ia.drop_frames(['lts_lda_feature_frame'])

# Copy the document x topic frame...
lts_lda_feature_frame = doc_mixes.copy(name='lts_lda_feature_frame')

# Create schema...
schema = map(lambda i: ('topic' + "_" + str(i), ia.float64), range(1, nTopics + 1))

# Create features frame...
lts_lda_feature_frame.add_columns(lambda row: (row['topic_probabilities']), schema)
lts_lda_feature_frame.drop_columns(['topic_probabilities'])

[=========================] 100.00% Tasks retries:0 Time 0:00:07
[=========================] 100.00% Tasks retries:0 Time 0:00:09
[=========================] 100.00% Tasks retries:0 Time 0:00:07


In [11]:
lts_lda_feature_frame.inspect()

[#]  VISID                 topic_1          topic_2          topic_3        
[0]  2d20146e46b49113d8c4  0.0518867115579  0.0502012089177  0.0475826049922
[1]  a88385757a83291e3974  0.0396291039776  0.0525130658564  0.0531770224448
[2]  b678e7336d72b8a58a8a  0.0502662958234  0.0507292049632   0.053816575841
[3]  068b3407012a9979ede3  0.0577279357939  0.0483479515598  0.0501342261662
[4]  7762e3ec6f2e76f7f7d0  0.0517545402405   0.054361824517  0.0484596102192
[5]  793b67d856c04409aab3  0.0464585811944  0.0491275913986  0.0470441404138
[6]  69c494556e28fffbadd9   0.056830934914  0.0532470074252  0.0501999235105
[7]  a53eed59a08255003a78  0.0532694252019  0.0487647319719  0.0458092453288
[8]  4e47c1cb427701cad79a   0.067189908422  0.0461926255776  0.0538418901593
[9]  89b44a87a94b82c448a6  0.0611161393671  0.0493166563365  0.0414604653475

[#]  topic_4          topic_5          topic_6          topic_7        
[0]  0.0472649748133  0.0490310397615  0.0577452872844  0.0545182204427
[1]  0.0

In [12]:
tutorial_inpat_model_lda = tutorial_inpat_model.join(lts_lda_feature_frame, 'VISID')

[=========================] 100.00% Tasks retries:0 Time 0:00:10


In [13]:
tutorial_inpat_model_lda.inspect(columns=[i for i in tutorial_inpat_model_lda.column_names if i != 'data_lines'])

[#]  PATID                 VISID               
[0]  86b314f2b8f649e09a25  bd62406626bdb419eb3c
[1]  c3ee6c31222a5c0004ce  7f6aee844984b9437bcb
[2]  1d9af07a834caa2cc3ab  49d6b4f4a2bdf68f8c62
[3]  818ddca22852f6f94f69  960262597b8a2dd19bae
[4]  04c4bf916059aea7340f  5590a4f169574de5b93a
[5]  1b40b5b4555e7675d578  df4b8e284086b26064ac
[6]  0a3a992c3a2287d9470f  4b202d09bf4bddc995d1
[7]  12b37095664c01df2454  66d27aba05a171b606a9
[8]  e05e07d93bdaae395d36  0401b8c91ae336b4a9b6
[9]  899c3ec20a4ac7bf00ab  f83959fecf8087e5a150

[#]  EVENT_ID                                  HEIGHT_INCHES  MARITAL_STATUS
[0]  86b314f2b8f649e09a25bd62406626bdb419eb3c  72.0078735352  W
[1]  c3ee6c31222a5c0004ce7f6aee844984b9437bcb  57.9921264648  S
[2]  1d9af07a834caa2cc3ab49d6b4f4a2bdf68f8c62  70.9842529297  M
[3]  818ddca22852f6f94f69960262597b8a2dd19bae  64.0157470703  M
[4]  04c4bf916059aea7340f5590a4f169574de5b93a  67.0078735352  M
[5]  1b40b5b4555e7675d578df4b8e284086b26064ac  59.9999961853  M
[6]  0a3a9

# Get gold-standard labels...

In [14]:
def get_gold_standard(row):
    """
    Function to compute nDays between discharge and admit.
    Returns GS label.
    """
    t2 = row["ADM_DATE"]
    t1 = row["DISCHARGE_DATE"]
    t1 = datetime.datetime.fromtimestamp(float(t1))
    t2 = datetime.datetime.fromtimestamp(float(t2))
    td = t2 - t1
    td_days = int(divmod(td.days, 60)[-1])
    if td_days <= 30:
        RF30 = "POSITIVE"
    else:
        RF30 = "NEGATIVE"
    return RF30

tutorial_inpat_model_lda.add_columns(get_gold_standard, ("RF30", str))

[=========================] 100.00% Tasks retries:0 Time 0:00:09


In [15]:
tutorial_inpat_model_lda.inspect(n=10, columns=[i for i in tutorial_inpat_model_lda.column_names if i in ['ADM_DATE', 'DISCHARGE_DATE', "RF30", "RF90"]])

[#]  ADM_DATE    DISCHARGE_DATE  RF30    
[0]  1557512280  1557828180      NEGATIVE
[1]  1557766800  1558266000      NEGATIVE
[2]  1558113600  1558613580      NEGATIVE
[3]  1557813180  1558035840      NEGATIVE
[4]  1557835500  1558025040      NEGATIVE
[5]  1557836460  1558869000      NEGATIVE
[6]  1557829740  1558186260      NEGATIVE
[7]  1557833280  1558016280      NEGATIVE
[8]  1557944400  1559234040      NEGATIVE
[9]  1557904680  1558108740      NEGATIVE

In [16]:
rf30_count = tutorial_inpat_model_lda.group_by('RF30', ia.agg.count)

[=========================] 100.00% Tasks retries:0 Time 0:00:02


In [17]:
rf30_count.inspect()

[#]  RF30      count
[0]  NEGATIVE   1500
[1]  POSITIVE     18

## Demo: Preparing a column for a classifier/algorithm--feature encoding train/test!

#### Demo: Encoding features

In [18]:
# Let's demonstrate this with a non-numeric column, the MARITAL_STATUS feature...
tutorial_inpat_model_lda.inspect(columns="MARITAL_STATUS")

[#]  MARITAL_STATUS
[0]  W
[1]  S
[2]  M
[3]  M
[4]  M
[5]  M
[6]  S
[7]  M
[8]  M
[9]  S

In [19]:
# Let's generate a summary table of the possible values here...
tutorial_inpat_model_lda_count = tutorial_inpat_model_lda.group_by("MARITAL_STATUS", ia.agg.count)

[=========================] 100.00% Tasks retries:0 Time 0:00:02


In [20]:
tutorial_inpat_model_lda_count.inspect()

[#]  MARITAL_STATUS  count
[0]  S                 637
[1]  M                 661
[2]  X                   1
[3]  A                  23
[4]  W                  91
[5]  D                 105

In [21]:
def numericalize_udf(in_val, dx):
        try:
            return_val = dx[in_val]
        except:
            return_val = 0
        return return_val
    
# ia.drop_frames("numerical_test")
# numerical_test = tutorial_inpat.copy(name='numerical_test')
# numerical_test_count = numerical_test.group_by('MARITAL_STATUS', ia.agg.count)

for col in ["MARITAL_STATUS"]:
    new_column_name = col + "_INT"
    f = tutorial_inpat_model_lda.download()
    d = f.to_dict()[col]
    rev_dx = dict((v, k) for k, v in d.iteritems())
    
    tutorial_inpat_model_lda.add_columns(lambda row: numericalize_udf(row[col], rev_dx), (new_column_name, ia.int32))

[=========================] 100.00% Tasks retries:0 Time 0:00:08


In [22]:
tutorial_inpat_model_lda.inspect()

[#]  PATID                 VISID               
[0]  86b314f2b8f649e09a25  bd62406626bdb419eb3c
[1]  c3ee6c31222a5c0004ce  7f6aee844984b9437bcb
[2]  1d9af07a834caa2cc3ab  49d6b4f4a2bdf68f8c62
[3]  818ddca22852f6f94f69  960262597b8a2dd19bae
[4]  04c4bf916059aea7340f  5590a4f169574de5b93a
[5]  1b40b5b4555e7675d578  df4b8e284086b26064ac
[6]  0a3a992c3a2287d9470f  4b202d09bf4bddc995d1
[7]  12b37095664c01df2454  66d27aba05a171b606a9
[8]  e05e07d93bdaae395d36  0401b8c91ae336b4a9b6
[9]  899c3ec20a4ac7bf00ab  f83959fecf8087e5a150

[#]  EVENT_ID                                  HEIGHT_INCHES  MARITAL_STATUS
[0]  86b314f2b8f649e09a25bd62406626bdb419eb3c  72.0078735352  W
[1]  c3ee6c31222a5c0004ce7f6aee844984b9437bcb  57.9921264648  S
[2]  1d9af07a834caa2cc3ab49d6b4f4a2bdf68f8c62  70.9842529297  M
[3]  818ddca22852f6f94f69960262597b8a2dd19bae  64.0157470703  M
[4]  04c4bf916059aea7340f5590a4f169574de5b93a  67.0078735352  M
[5]  1b40b5b4555e7675d578df4b8e284086b26064ac  59.9999961853  M
[6]  0a3a9

### Partition into train/test

In [23]:
PATIDS = tutorial_inpat_model_lda.group_by("PATID", ia.agg.count)
PATIDSdl = PATIDS.download(n=PATIDS.row_count, columns='PATID')
unique_patids = list(set(list(PATIDSdl['PATID'])))
random.shuffle(unique_patids)
test_patids = unique_patids[0:int(len(unique_patids) * 0.10)]
train_patids = [i for i in unique_patids if i not in test_patids]


def label_train_test(row, test_list=test_patids, train_list=train_patids):
    if row['PATID'] in test_patids:
        return "TEST"
    else:
        return "TRAIN"
    

tutorial_inpat_model_lda.add_columns(label_train_test, ("DATASET", str))

[=========================] 100.00% Tasks retries:0 Time 0:00:02
[=========================] 100.00% Tasks retries:0 Time 0:00:16


In [24]:
len(train_patids)

1334

In [25]:
len(test_patids)

148

In [26]:
tutorial_inpat_model_lda.inspect(columns=[i for i in tutorial_inpat_model_lda.column_names if i != 'data_lines'])

[#]  PATID                 VISID               
[0]  86b314f2b8f649e09a25  bd62406626bdb419eb3c
[1]  c3ee6c31222a5c0004ce  7f6aee844984b9437bcb
[2]  1d9af07a834caa2cc3ab  49d6b4f4a2bdf68f8c62
[3]  818ddca22852f6f94f69  960262597b8a2dd19bae
[4]  04c4bf916059aea7340f  5590a4f169574de5b93a
[5]  1b40b5b4555e7675d578  df4b8e284086b26064ac
[6]  0a3a992c3a2287d9470f  4b202d09bf4bddc995d1
[7]  12b37095664c01df2454  66d27aba05a171b606a9
[8]  e05e07d93bdaae395d36  0401b8c91ae336b4a9b6
[9]  899c3ec20a4ac7bf00ab  f83959fecf8087e5a150

[#]  EVENT_ID                                  HEIGHT_INCHES  MARITAL_STATUS
[0]  86b314f2b8f649e09a25bd62406626bdb419eb3c  72.0078735352  W
[1]  c3ee6c31222a5c0004ce7f6aee844984b9437bcb  57.9921264648  S
[2]  1d9af07a834caa2cc3ab49d6b4f4a2bdf68f8c62  70.9842529297  M
[3]  818ddca22852f6f94f69960262597b8a2dd19bae  64.0157470703  M
[4]  04c4bf916059aea7340f5590a4f169574de5b93a  67.0078735352  M
[5]  1b40b5b4555e7675d578df4b8e284086b26064ac  59.9999961853  M
[6]  0a3a9

## Train Random Forest

In [27]:
trainframe = tutorial_inpat_model_lda.copy()
trainframe.filter(lambda row: row['DATASET'] == "TRAIN")
testframe = tutorial_inpat_model_lda.copy()
testframe.filter(lambda row: row['DATASET'] == "TEST")

[=========================] 100.00% Tasks retries:0 Time 0:00:12
[=========================] 100.00% Tasks retries:0 Time 0:00:08
[=========================] 100.00% Tasks retries:0 Time 0:00:11
[=========================] 100.00% Tasks retries:0 Time 0:00:07


In [28]:
train.column_names

NameError: name 'train' is not defined

In [ ]:
rf_model_tutorial = ia.RandomForestClassifierModel('rf_model_tutorial2')

In [ ]:
# train.column_names
rf_model_tutorial

In [ ]:
def rf30_str(row):
    if row['RF30'] == "POSITIVE":
        return 1
    else:
        return 0

trainframe.add_columns(rf30_str, ("RF30STR", ia.float32))
testframe.add_columns(rf30_str, ("RF30STR", ia.float32))

In [ ]:
rf_model_tutorial_results = rf_model_tutorial.train(frame=trainframe, 
                                                    label_column="RF30STR", 
                                                    observation_columns=["topic_1", "topic_2"], 
                                                    num_classes=2, 
                                                    impurity='gini', 
                                                    max_depth=4, 
                                                    seed=01001000
                                                   )

In [ ]:
# test the model
x = rf_model_tutorial.test(testframe, "RF30STR")

In [ ]:
x

In [ ]:
x = m.publish()